In [7]:
from pyMSOO.MFEA.model import SBSGA
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.DimensionAwareStrategy import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 
from pyMSOO.utils.MultiRun.RunMultiBenchmark import * 

In [2]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(1)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()

In [3]:
baseModel = SBSGA.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy=DaS_strategy(eta= 3)
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.5, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Exception ignored on calling ctypes callback function: <function ExecutionEngine._raw_object_cache_notify at 0x00000237AED459D0>
Traceback (most recent call last):
  File "f:\Anaconda\Anaconda\envs\MFEA\lib\site-packages\llvmlite\binding\executionengine.py", line 171, in _raw_object_cache_notify
    def _raw_object_cache_notify(self, data):
KeyboardInterrupt: 


In [3]:
import numpy as np

# Define a matrix with a very small determinant
A = np.array([[1, 2, 3, 0], [4, 5, 6, 0], [0, 7, 8, 9.00001]])

# Calculate the determinant
det_A = np.linalg.det(A)

print(det_A)


LinAlgError: Last 2 dimensions of the array must be square

### MUltitime Model

In [10]:
baseModel = MultiBenchmark(model= SBSGA, ls_benchmark=[tasks], ls_IndClass=[IndClass], name_benchmark = ['CEC17'])
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy= NoDaS(),
)
solve = baseModel.fit(
    nb_generations = 1000,nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

In [11]:
baseModel.run(
    nb_run= 1,
    save_path= './RESULTS/SBSGA.mso'
)

Seed: 0 -- Time: 01m 30.67s  100 % [====================>]  Cost: 2.24E-06  2.82E-07  2.13E-10  0.00E+00  3.60E-04  1.65E-04  1.84E-03  6.39E-04  4.14E-08  1.82E+01  ,  
END!
DONE!
Saved
